In [1]:
import numpy as np
from helpers import *
from implementations import *

In [2]:
(y, ox, ids) = load_csv_data('data/train.csv')

(_, otest, testid) = load_csv_data('data/test.csv')

In [3]:
x = ox.copy()
test = otest.copy()

In [4]:
(y.shape, x.shape, ids.shape)

((250000,), (250000, 30), (250000,))

In [5]:
x[1]

array([ 1.60937e+02,  6.87680e+01,  1.03235e+02,  4.81460e+01,
       -9.99000e+02, -9.99000e+02, -9.99000e+02,  3.47300e+00,
        2.07800e+00,  1.25157e+02,  8.79000e-01,  1.41400e+00,
       -9.99000e+02,  4.20140e+01,  2.03900e+00, -3.01100e+00,
        3.69180e+01,  5.01000e-01,  1.03000e-01,  4.47040e+01,
       -1.91600e+00,  1.64546e+02,  1.00000e+00,  4.62260e+01,
        7.25000e-01,  1.15800e+00, -9.99000e+02, -9.99000e+02,
       -9.99000e+02,  4.62260e+01])

In [6]:
#remove 999s, replacing by mean of column
x[np.where(x == -999)] = np.NaN
col_mean = np.nanmean(x, axis=0)
inds = np.where(np.isnan(x))
x[inds] = np.take(col_mean, inds[1])

In [7]:
x[1]

array([ 1.60937000e+02,  6.87680000e+01,  1.03235000e+02,  4.81460000e+01,
        2.40373503e+00,  3.71783360e+02, -8.21688171e-01,  3.47300000e+00,
        2.07800000e+00,  1.25157000e+02,  8.79000000e-01,  1.41400000e+00,
        4.58289801e-01,  4.20140000e+01,  2.03900000e+00, -3.01100000e+00,
        3.69180000e+01,  5.01000000e-01,  1.03000000e-01,  4.47040000e+01,
       -1.91600000e+00,  1.64546000e+02,  1.00000000e+00,  4.62260000e+01,
        7.25000000e-01,  1.15800000e+00,  5.76794744e+01, -1.18452642e-02,
       -1.58228913e-03,  4.62260000e+01])

In [8]:
test[1]

array([ 1.06398e+02,  6.74900e+01,  8.79490e+01,  4.99940e+01,
       -9.99000e+02, -9.99000e+02, -9.99000e+02,  2.04800e+00,
        2.67900e+00,  1.32865e+02,  1.77700e+00, -1.20400e+00,
       -9.99000e+02,  3.07160e+01, -1.78400e+00,  3.05400e+00,
        5.45740e+01, -1.69000e-01,  1.79500e+00,  2.10930e+01,
       -1.13800e+00,  1.76251e+02,  1.00000e+00,  4.75750e+01,
       -5.53000e-01, -8.49000e-01, -9.99000e+02, -9.99000e+02,
       -9.99000e+02,  4.75750e+01])

In [9]:
test[np.where(test == -999)] = np.NaN
inds = np.where(np.isnan(test))
test[inds] = np.take(col_mean, inds[1])

In [10]:
test[1]

array([ 1.06398000e+02,  6.74900000e+01,  8.79490000e+01,  4.99940000e+01,
        2.40373503e+00,  3.71783360e+02, -8.21688171e-01,  2.04800000e+00,
        2.67900000e+00,  1.32865000e+02,  1.77700000e+00, -1.20400000e+00,
        4.58289801e-01,  3.07160000e+01, -1.78400000e+00,  3.05400000e+00,
        5.45740000e+01, -1.69000000e-01,  1.79500000e+00,  2.10930000e+01,
       -1.13800000e+00,  1.76251000e+02,  1.00000000e+00,  4.75750000e+01,
       -5.53000000e-01, -8.49000000e-01,  5.76794744e+01, -1.18452642e-02,
       -1.58228913e-03,  4.75750000e+01])

In [11]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree.
    
    Args:
        x: numpy array of shape (N,D)
        degree: integer.
        
    Returns:
        poly: numpy array of shape (N,D*d+1)
    """    
    N = x.shape[0]
    D = x.shape[1]
    #poly_base = np.zeros((N, D*degree + 1))
    poly_base = np.ones((N,1))
    poly_base = np.hstack((poly_base, x.copy()))
    for i in range(degree-1):
        range_start = 1 + i*D
        range_stop = 1 + (i+1)*D
        next_power = poly_base[:,range_start:range_stop]*x
        poly_base = np.hstack((poly_base, next_power))
    
    return poly_base

def expand_features(x):
    return build_poly(x, 6)

In [12]:
x = expand_features(x)
test = expand_features(test)

NameError: name 'trainmean' is not defined

In [14]:
#standardize
trainmean = x.mean(axis = 0)
trainstd  = x.std(axis = 0)

#handle the case of the column of ones with std 0 : modify nothing
trainmean[0] = 0
trainstd[0]  = 1.0

x = (x-trainmean)/trainstd
test = (test - trainmean)/trainstd

In [15]:
(w, loss) = least_squares(y, x)

In [16]:
def makeFinalPrediction(test, w):
    res = test @ w
    res[np.where(res < 0)] = -1
    res[np.where(res >= 0)] = 1
    create_csv_submission(testid, res, "zlatan3.csv")

In [17]:
makeFinalPrediction(test, w)